# Fractal Generator - Growth #
### python -file for Fractal calculations Cuda accelerated ###

JV - 2025 / 1

'Generates' Fractal pictures by using openCV  
Avoiding to use Matplotlib functions due to slow speed and high memory demand 
Image Matrices / Numpy's :
CMAP_BUF : color space   
FRAC / NFRAC: fractal array of floats64 with relative values between 0 and 1  
FOI : Field of Interest plot, marks points of interest where to zoom in (max unique points)
IMAGE_BUF: Image buffer, definition kept out of functions to speed up . 

Using @Jit NUMBA compiler options for CPU acceleration  - install the cuda tookit 'conda install cudatoolkit' 
Check your OpenCV video possibilities, this depends on your installed system (Raspi / Windowsa / Linux etc)
All coordinates are used in Y-X (like Row-Column) to avoid messing up your intepretations between graph and arrays :)

# 2 Steps:
1 Generate start point and color space - (use a-z keys for finding growing area's)<br>
2 Let is grow : Video generation

### Ad Step1 : Click and zoom functions:
Mouse: click to zoom in or out (left/right click).
* Color shuffle:  x = Grayscale,   c = New Color map ,   v = inverse color space, f = filer on/of
* Manual color panning:  red u-j, green i-k, blue o-l 
* \[-\] shift colors
* a-z PAN 3rd DIMENSION grower < ----------------------------------------
* s = save image
* r = reset color space
* q = QUIT

See def mandel(x, y, max) function to choose different complex functions

https://github.com/javos65/Cuda-Fractal

## Load Libraries and global variables

In [6]:
import os 
import cv2
from datetime import datetime
import numba 
from numba import jit
import numpy as np
import scipy as sc
import math
import time
from numpy.random import default_rng
rng = default_rng()
import cmath
import random
from numba import cuda
cuda.select_device(0)
global IMAGENR; IMAGENR=0
global THREADS; THREADS=800      # RTX3080 max threads
global CLICKL
CLICKL=np.array(np.zeros((1,2), dtype=np.int32))
global CLICKR
CLICKR=np.array(np.zeros((1,2), dtype=np.int32))
#global variable for color map generator
global Gf
global Rf
global Bf
global Zf
global If
global Ff
Gf=2.0;Rf=2.0;Bf=2.0;Zf=2.0;If=0; Ff=0; # Color spacve parameters

## Function Declarations

In [201]:
# Fractal kernal routines for Mandelbrot
def calculate_mandelbrot_imageC(Ny,Nx,Zz,itmax,center,scl,IMG) :                 
    x1=center[1]-Nx/(2*scl); x2=center[1]+Nx/(2*scl)
    y1=center[0]-Ny/(2*scl); y2=center[0]+Ny/(2*scl)    
    warp = 32
    blx = warp; bly = math.ceil(THREADS/warp)                 # prepare Cuda RTX3080: warp = 32, Threads = 1024
    grx=math.ceil(Nx/blx); gry=math.ceil(Ny/bly)
    blockdim = (blx, bly) ; griddim = (grx,gry)
    d_image = cuda.to_device(IMG)                         #send FRAC image to Device
    mandel_kernel[griddim, blockdim](x1, x2, y1, y2, Zz , d_image, itmax) # call cuda function
    IMG = d_image.copy_to_host()                          #copy image back to host
    return IMG

@cuda.jit(device=True)
def mandel(x, y, u, max):
    c = complex(y, x) ; 
    z= complex(0,0); 
    
    for i in range(max):
################  MANDEKBROT FUNCTIONS #############
        z1= z*z - c
        z2= z*z + u

        z=z1*z2+c
        #z = (z-z1)*(z2+z)          # amoebe 0
        #z = z1*z2          # amoebe 1
        #z = (z-z1)*(z-z2) + c          # amoebe 2 
        #z = (z-z1)*(z-z2) - c          # amoebe 3
        #z = z*z+c
################  MANDEKBROT FUNCTIONS #############        
        if ((z.real)**2 + (z.imag)**2)  >= 4:
            return i
    return max
       
@cuda.jit
def mandel_kernel(min_x, max_x, min_y, max_y,max_z, image, iters):
  height = image.shape[0]
  width = image.shape[1]

  pixel_size_x = (max_x - min_x) / width
  pixel_size_y = (max_y - min_y) / height

  startX = cuda.blockDim.x * cuda.blockIdx.x + cuda.threadIdx.x
  startY = cuda.blockDim.y * cuda.blockIdx.y + cuda.threadIdx.y
  gridX = cuda.gridDim.x * cuda.blockDim.x;
  gridY = cuda.gridDim.y * cuda.blockDim.y;

  for x in range(startX, width, gridX):
    real = min_x + x * pixel_size_x
    for y in range(startY, height, gridY):
      imag = min_y + y * pixel_size_y 
      image[y, x] = mandel(real, imag, max_z, iters)/iters
        

# Field of interest calculator : edge detector in 5x5 pixel grid
def calculate_foiC(M,foi):                                            # Callable function
    hy = M.shape[0]
    wx = M.shape[1]
    warp = 32
    blx = warp; bly = math.ceil(THREADS/warp)         # optimum is max 32 /512 threads per Block : gtx1080, 32 /1024 for RTx 3080
    grx=math.ceil(wx/blx); gry=math.ceil(hy/bly)  # calculate grid
    blockdim = (blx, bly) ; griddim = (grx,gry)   # define block and grid
    m_image = cuda.to_device(M)
    f_image = cuda.to_device(foi)
    foi_kernel[griddim, blockdim](m_image,f_image)
    foi=f_image.copy_to_host()
    #M=m_image.copy_to_host()
    return foi   

@cuda.jit
def foi_kernel(fimagein, fimageout):
  height = fimagein.shape[0]
  width = fimagein.shape[1]

  startX = cuda.blockDim.x * cuda.blockIdx.x + cuda.threadIdx.x
  startY = cuda.blockDim.y * cuda.blockIdx.y + cuda.threadIdx.y
  gridX = cuda.gridDim.x * cuda.blockDim.x;
  gridY = cuda.gridDim.y * cuda.blockDim.y;

  for x in range(startX, width, gridX):
    for y in range(startY, height, gridY):
      p = fimagein[y,x] ;dev=0
      for t in range (-2,3,1) :
        for u in range (-2,3,1) :
            if y+t>=0 and y+t<height and x+u>=0 and x+u<width :
                dev = dev + (p-fimagein[y+t,x+u])*(p-fimagein[y+t,x+u])
      fimageout[y, x] = dev/25  # this should be a square root, but for foi indication the sum^2 is enough


    
            
# Map fractal array into CV2-image map with R-G-B coding using Color Map
def cmap_cvimageC(fr,cv,cmap) :                                #callable function
    hy = fr.shape[0]
    wx = fr.shape[1]
    cd= cmap.shape[1]
    warp = 32
    blx = warp; bly = math.ceil(THREADS/warp); blz=1         # optimum is max 512 threads per Block : gtx1080 , 1024 RTX3080
    grx=math.ceil(wx/blx); gry=math.ceil(hy/bly); grz= cd  # calculate grid
    blockdim = (blx, bly, blz) ; griddim = (grx,gry,grz)   # define block and grid
    f_image = cuda.to_device(fr)
    c_image = cuda.to_device(cv)
    m_image = cuda.to_device(cmap)
    cmap_kernel[griddim, blockdim](f_image,c_image,m_image)
    #cmap=m_image.copy_to_host()
    cv=c_image.copy_to_host()
    #fr=f_image.copy_to_host()    
    return cv

    
@cuda.jit
def cmap_kernel(frac,cvimg,cmap) :
    hy = frac.shape[0]
    wx = frac.shape[1]
    rr = cmap.shape[0]
    cd = cmap.shape[1]
    startX = cuda.blockDim.x * cuda.blockIdx.x + cuda.threadIdx.x
    startY = cuda.blockDim.y * cuda.blockIdx.y + cuda.threadIdx.y
    startZ = cuda.blockDim.z * cuda.blockIdx.z + cuda.threadIdx.z
    gridX = cuda.gridDim.x * cuda.blockDim.x;
    gridY = cuda.gridDim.y * cuda.blockDim.y;
    gridZ = cuda.gridDim.z * cuda.blockDim.z;
    for x in range(startX, wx, gridX):
        for y in range(startY, hy, gridY):
            index = int(frac[y,x]*(rr-1))
            index=index%rr                      #protection to avoid faul indexes
            for z in range(startZ, cd, gridZ):
                cvimg[y,x,z]=cmap[index,z]

# Filter Fractal array
def filter_fracC(fr,cv,fmap) :                                #callable function
    hy = fr.shape[0]
    wx = fr.shape[1]
    cd= fmap.shape[1]
    warp = 32
    blx = warp; bly = math.ceil(THREADS/warp); blz=1         # optimum is max 512 threads per Block : gtx1080 , 1024 RTX3080
    grx=math.ceil(wx/blx); gry=math.ceil(hy/bly); grz= cd  # calculate grid
    blockdim = (blx, bly, blz) ; griddim = (grx,gry,grz)   # define block and grid
    f_image = cuda.to_device(fr)
    c_image = cuda.to_device(cv)
    f_map = cuda.to_device(fmap)
    filter_kernel[griddim, blockdim](f_image,c_image,f_map)
    #cmap=m_image.copy_to_host()
    cv=c_image.copy_to_host()
    #fr=f_image.copy_to_host()    
    return cv
   
    
@cuda.jit
def filter_kernel(frac,cvimg,fmap) :
    hy = frac.shape[0]
    wx = frac.shape[1]
    fy = fmap.shape[0]
    fx = fmap.shape[1]
    tt=0;uu=0;
    startX = cuda.blockDim.x * cuda.blockIdx.x + cuda.threadIdx.x
    startY = cuda.blockDim.y * cuda.blockIdx.y + cuda.threadIdx.y
    startZ = cuda.blockDim.z * cuda.blockIdx.z + cuda.threadIdx.z
    gridX = cuda.gridDim.x * cuda.blockDim.x;
    gridY = cuda.gridDim.y * cuda.blockDim.y;
    gridZ = cuda.gridDim.z * cuda.blockDim.z;
    for x in range(startX, wx, gridX):
        for y in range(startY, hy, gridY):
          p = frac[y,x]; 
          av=0; wg=0
          if p<0.2 or p>0.80:
            for t in range (0,fy,1) :
                tt = int(t-(fy-1)/2);
                for u in range (0,fx,1) :
                    uu= int(u-(fx-1)/2);
                    if (y+tt)>=0 and (y+tt)<hy and (x+uu)>=0 and (x+uu)<wx :
                          wg = wg + fmap[t,u];
                          av = av + frac[y+tt,x+uu]*fmap[t,u];
            cvimg[y,x] = av/wg  # this should be a square root, but for foi indication the sum^2 is enough
          else:
            cvimg[y,x] = p;
            
               
                
                
# calculate zoom point by splitting most interesting point of the FOI into an array, then select point, calculate new center.
def calculate_zoompoint(foi,centero,resy,resx, scale, slce) :
    foi_r = np.where(foi > np.max(foi)/5 )  # results in 2 1 dimensionla arrays  col:row
    s=np.array([0])
    if slce == 0 : s = np.random.uniform(0,foi_r[:][0].size-1,2)
    else : s[0]=  foi_r[:][0].size*(slce%1)
    xindex=foi_r[1][int(s[0])]
    yindex=foi_r[0][int(s[0])]
    centern= ( centero[0] + (yindex-resy/2)/scale,  centero[1] + (xindex-resx/2)/scale, yindex,xindex)
    return centern # center results in 4 points: real y/x: -> fractal points and mapped y/x -> indexed points of the array

def calculate_zoompointXY(centero,y,x,resy,resx, scale) :
    centern= ( centero[0] + (y-resy/2)/scale,  centero[1] + (x-resx/2)/scale, y,x)
    return centern # center results in 4 points: real y/x: -> fractal points and mapped y/x -> indexed points of the array



#  Complex function to generate 16bit ColorMaps for Fractals by sinus variations. 
#  Seed =0 : Grayscale, Seed = odd : white color Seed = even: black color
def create_cmap(buf,seed) :
    global Rf; global Bf; 
    global Gf; global Zf ; global If
    r=0;g=0;b=0;inv=0;
    color_bits = 256-1; # using 16 bits per color
    x=buf.shape[0]
    if seed == 0 :         # seed 0 is Gray scale
        r=4;g=4;b=4;inv=0;
    else : 
        if seed == 99 : # color frequency by keyboard
            #do
            r=Rf
            g=Gf
            b=Bf
            inv=If
        else :
            # other seed is random: 
            r= random.uniform(0.65,3.5)
            g= random.uniform(0.65,3.5)
            b= random.uniform(0.65,3.5)
            Rf=r;Bf=b;Gf=g;If=inv;
    for i in range(0,x,1):
        if i*1*math.pi/x < 1*math.pi/6 or i*1*math.pi/x > 5*math.pi/6  : 
            l1= math.cos(i*3*math.pi/x-math.pi/2)  # lightness mask =  halve 4*sine, cut off at 1
        else :  l1= 0.75+ math.cos(i*3*math.pi/x-math.pi/2)/4  # lightness mask =  halve 4*sine, cut off at 1
        if l1>1 : l1=1

        buf[x-i-1][0]= abs( color_bits*inv -int (color_bits*l1*(1+math.sin(i*r*math.pi/x-Zf*math.pi/2))/2  )) # R Channel
        buf[x-i-1][1]= abs( color_bits*inv -int (color_bits*l1*(1+math.sin(i*g*math.pi/x-Zf*math.pi/2))/2  )) # G Channel
        buf[x-i-1][2]= abs( color_bits*inv -int (color_bits*l1*(1+math.sin(i*b*math.pi/x-Zf*math.pi/2))/2  )) # B Channel
        #buf[x-i-1][3]=1
        r=r*1.001;g=g*1.001;b=b*1.001
        
    buf.reshape(x,3)
    return buf  

# function to display the coordinates of of the points clicked on the image
def click_event(event, x, y, flags, params): 
    global CLICKL
    global CLICKR
    # checking for left mouse clicks 
    if event == cv2.EVENT_LBUTTONDOWN: 
        CLICKL=(y,x)
    if event == cv2.EVENT_RBUTTONDOWN: 
        CLICKR=(y,x)

## Start Conditions

In [173]:
#Start Conditions
iterate_max = 256 ; center = (-0.25,0, 0,0); scale = 200.0; Ny=720; Nx=1600; Nz=480;Zz=-1.06  # start parameters for iteration max and centerpoint : Y-X coordinates !!, Nx and Ny are image resolution
Gf=2.3;Rf=2.1;Bf=1.9;Zf=1.0;If=0; Ff=0;
FMAP_BUF =  np.matrix ([
      [0.0, 0.0, 0.1, 0.0, 0.0],
      [0.0, 0.2, 0.4, 0.2, 0.0],
      [0.1, 0.4, 0.0, 0.4, 0.1],
      [0.0, 0.2, 0.4, 0.2, 0.0],
      [0.0, 0.0, 0.1, 0.0, 0.0]],dtype=np.float64)

MATRIX = np.array(np.zeros((Ny,Nx,Nz), dtype=np.float64)) 
BASE = np.array(np.zeros((Ny,Nx), dtype=np.float64))   # define fractal array global, not in function, float64 type
FOI = np.array(np.zeros((Ny,Nx), dtype=np.float64))                                          # define FOI image, same size
IMAGE_BUF = np.zeros((Ny, Nx,3), dtype = np.uint8)      # define Color image buffer
CMAP_BUF =  np.zeros((iterate_max,3), dtype = np.uint8) # define Color Map 16 bit type, x 3 (RGB) - no alfa channel
CMAP_BUF = create_cmap(CMAP_BUF,2)                       # create a color map
C=CMAP_BUF.reshape(1,iterate_max,3)
C=np.repeat(C,32,axis=0)
cv2.imwrite("CMAP.png", C)

t= datetime.now()
FRAC = calculate_mandelbrot_imageC(Ny,Nx,Zz,iterate_max,center,scale,BASE) # calculate mandelbrot
NFRAC = FRAC.copy();
NFRAC=filter_fracC(FRAC,NFRAC,FMAP_BUF)
frame=cmap_cvimageC(NFRAC,IMAGE_BUF,CMAP_BUF)
t=datetime.now()-t
print(t.microseconds)
frame=cmap_cvimageC(FRAC,IMAGE_BUF,CMAP_BUF)
cv2.imwrite("MandelOrg.png", frame) 
 

247065


True

## Step1: Click and Zoom Fractal Growth

In [197]:
#FMAP_BUF, CMAP_BUF and FRAC already defined = mandel set
CLICKL=(0,0)
CLICKR=(0,0)
nr = 0
while 1 :
    cuda.synchronize()
    FRAC = calculate_mandelbrot_imageC(Ny,Nx,Zz,iterate_max,center,scale,BASE) # calculate new mandelbrot Set
    if Ff ==1 : 
        NFRAC=filter_fracC(FRAC,NFRAC,FMAP_BUF) ; # filter
        frame=cmap_cvimageC(NFRAC,IMAGE_BUF,CMAP_BUF)
    else : frame=cmap_cvimageC(FRAC,IMAGE_BUF,CMAP_BUF)  
    cv2.imshow('image',frame)
    
    #C=CMAP_BUF.reshape(1,iterate_max,3)
    #C=np.repeat(C,32,axis=0)
    #cv2.imshow('cmap',C)
    
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"): # if the `q` key was pressed, break from the loop
        break 
    if key == ord("s"): # if the `s pressed, save picture in PNG format
        cv2.imwrite("MandelZoom"+str(IMAGENR)+".png", frame)  
        IMAGENR=IMAGENR+1
    if key == ord("z"): # if the `z` Pan 3rd dimension < ----------------------------------------------------------------- !!!
        Zz = Zz + 1/scale       
    if key == ord("a"): # if the `a` Pan 3rd dimension  < --------------------------------------------------------------- !!!
        Zz = Zz - 1/scale               
    if key == ord("c"): # if the `c` create new color map black
        CMAP_BUF = create_cmap(CMAP_BUF,1)
    if key == ord("v"): # if the `v` toggle inversed map
        if If == 0: If=1;
        else : If=0;
        CMAP_BUF = create_cmap(CMAP_BUF,99)
    if key == ord("f"): # if the `v` toggle filter on of
        if Ff == 0: Ff=1;
        else : Ff=0;        
        CMAP_BUF = create_cmap(CMAP_BUF,99)
    if key == ord("x"): # if the `x` set grayscale
        Gf=Rf;Bf=Rf; CMAP_BUF = create_cmap(CMAP_BUF,0)
    if key == ord("u"): # if the `u` pan color map red to higher freq
        Rf=Rf*1.05; CMAP_BUF = create_cmap(CMAP_BUF,99)
    if key == ord("j"): # if the `j` pan color map red to lower freq
        Rf=Rf*0.95; CMAP_BUF = create_cmap(CMAP_BUF,99)
    if key == ord("i"): # if the `i` pan colormap green to higher freq
        Gf=Gf*1.05; CMAP_BUF = create_cmap(CMAP_BUF,99)
    if key == ord("k"): # if the `k` pan colormap green to lower freq
        Gf=Gf*0.95; CMAP_BUF = create_cmap(CMAP_BUF,99)
    if key == ord("o"): # if the `o` pan color map blue to higher freq
        Bf=Bf*1.05; CMAP_BUF = create_cmap(CMAP_BUF,99)
    if key == ord("l"): # if the `l` pan color map blue to lower freq
        Bf=Bf*0.95; CMAP_BUF = create_cmap(CMAP_BUF,99)
    if key == ord("["): # if the `[` rotate colormap left
        Zf=Zf*1.02; CMAP_BUF = create_cmap(CMAP_BUF,99)
    if key == ord("]"): # if the `]'  srotate color map right
        Zf=Zf*0.98; CMAP_BUF = create_cmap(CMAP_BUF,99)
    if key == ord("r"): # reset all color parameters
        Gf=2.3;Rf=2.1;Bf=1.9;Zf=1.0;If=0; Ff=0; CMAP_BUF = create_cmap(CMAP_BUF,99)        
               
    cv2.setMouseCallback('image', click_event)     
    if CLICKL != (0,0) :
        center = calculate_zoompointXY(center,CLICKL[0],CLICKL[1],Ny,Nx,scale)
        #print(CLICK,center)
        scale = scale*3;  #iterate_max = iterate_max*1.1   # slowly zoom in
        CLICKL=(0,0);
    if CLICKR != (0,0) :
        center = calculate_zoompointXY(center,CLICKR[0],CLICKR[1],Ny,Nx,scale)
        #print(CLICK,center)
        scale = scale/3;  #iterate_max = iterate_max/1.1   # slowly zoom out
        CLICKR=(0,0);    
        
cv2.destroyAllWindows() 

## Step2: Fractal Growth Video

In [199]:
frame=cmap_cvimageC(FRAC,IMAGE_BUF,CMAP_BUF) # first FRAC frame

# Define the codec and create VideoWriter object.The output is stored in 'outpy.mp4' file.
out= cv2.VideoWriter('Fractal3D.mp4', cv2.VideoWriter_fourcc(*'DIVX'), 30, (Nx,Ny))
#out = cv2.VideoWriter('Fractal3D.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 40, (Nx,Ny))
#out = cv2.VideoWriter('fraxvid.avi', cv2.VideoWriter_fourcc('X','V','I','D'), 40, (Nx,Ny))

Zo=Zz 
for j in range(0,Nz,1) :
    cuda.synchronize()
    Zz= Zo + (j-Nz/2)/(scale)
    NFRAC = calculate_mandelbrot_imageC(Ny,Nx,Zz,iterate_max,center,scale,BASE) # calculate mandelbrot
    #FRAC =filter_fracC(NFRAC,FRAC,FMAP_BUF)
    MATRIX[:,:,j]=NFRAC # map frame into Matrix
    frame=cmap_cvimageC(NFRAC,IMAGE_BUF,CMAP_BUF)
    cv2.imshow('frame2',frame)        
    out.write(frame)
    key = cv2.waitKey(1) & 0xFF
    if key == ord("q"): # if the `q` key was pressed, break from the loop
        break        
out.release()
cv2.destroyAllWindows() 

In [5]:
cv2.destroyAllWindows() 

## WRITE Frame to 3D pointcloud XYZrgb TXT-FILE ##

In [27]:
l=[]
colorbits = 256-1 # color bits of the CMAP matrix : 16 bits
rr = CMAP_BUF.shape[0]
for i in range(0,Ny,1):
    for j in range(0,Nx,1) :
        fr= FRAC[i, j]
        index = fr*(rr-1) ;
        index = int(index%rr)  # color index 
        red = int(CMAP_BUF[index,0]*255/colorbits)
        green = int(CMAP_BUF[index,1]*255/colorbits)
        blue = int(CMAP_BUF[index,2]*255/colorbits)
        if i==Ny-1 and j==Nx-1 : l.append('%d,%d,%d,%d,%d,%d' %(i, j, 100*fr, red , green, blue ) )
        else : l.append('%d,%d,%d,%d,%d,%d\n' %(i, j, 100*fr, red , green, blue ) )

with open('fractal.xyz', 'w') as f:
    f.write("".join(l))
f.close()

